<h1> Excersises 8 - Programming continues </h1>

<h3> Modyfing data </h3>

Last time we have loaded data layers, did selection on them and we accessed attributes of vector and raster layers. 

Today we will edit parameters and values of object of vector layers.

We will work on exemplary vector layer. Reminder: to access currently loaded layer:

In [ ]:
#Has to be selected in the layer list window!
layer = iface.activeLayer()

Changes and modifications are not visible until refreshed. To refresh a layer and to see changes on the map canvas do:

In [ ]:
#refreshing
layer.triggerRepaint() #by layer
#OR
iface.mapCanvas().refresh() #by interface

<h3> Adding new objects </h3>

We assume working on a vector layer with LineString type of data (lines). Such a file was attachet to excersise files.

Each object (QgsFeature) of the vector layer (QgsVectorLayer) has:
<ul>
    <li> Attributes (QgsField) </li>
    <li> Geometry (QgsGeometry) </li>
</ul>

When creating new object both have to be set.

First we create an empty new object:

In [ ]:
feat = QgsFeature(warstwa.fields()) #initilizing new QgsFeature object, fields of a layer are the parametres

It is not assigned to the layer yet, we will do it later. First it has to be given attributes' values and geometry:

In [ ]:
feat.setAttributes([0, 60]) #adding attributes, this layer has to have attributes of int type
#OR
feat.setAttribute(0,60) #if we are adding only one row

Now we create geometry:

In [ ]:
#Line from two points. First we create two objects of QgsPoint type - see description below:
line_start = QgsPoint(15.74,50.89)
line_end = QgsPoint(15.73,50.91)

QgsGeometry is a class of geometry, which can be points, line or polygons. Most often geometry is build from series of points (all the geometries are from points - points, lines and polygons). For that we use([new class here!]) QgsPoint:

    QgsPoint(x,y)

After, we use on of the methods that create QgsGeometry from QgsPoints from __geometry__, that is:
 
    fromPolyline, fromPolygonXY, fromPointXY or fromWKT - that is WKT type of data

In [ ]:
feat.setGeometry(QgsGeometry.fromPolyline([line_start,line_end]))

And we add it to the layer now. For that we are using the dataProvider class, which is always used for all the operations of adding, deleting and editing objects of a layer (more about it below):

In [ ]:
layer.dataProvider().addFeatures([feat]) #we add the feature to the layer

Besides QgsPoints, geometry can be build from WKT, which is much easier, because EVERY type of can be written in that file format.

<h3> WKT - Well-known text representation of geometry </h3>

It allows to define many types of geometry, including:

    Point (single), MultiPoint (collection of Points),
    LineString (single), MultiLineString (collection of Lines),
    Polygon (singlle), MultiPolygon (collection of Polygons).

Definition of a geometry:

    Nazwa (x y)

Point:

    POINT (10 30) - Point with coordinates x=10 i y = 30

Line:

    LINESTRING (30 10, 10 30) - a simple straight line where first vertex: X = 30, Y = 10 second vertex: X = 10, Y = 30

    LINESTRING (30 10, 10 30, 40 40) - line consisting of three points.
    
Polygon:

    POLYGON ((30 10, 40 40, 20 20, 10 20, 30 10))
    
    POLYGON ((35 10, 45 45, 15 40, 10 20, 35 10), (20 30, 35 35, 30 20, 20 30))
    
    The first and the last points is repeating because polygon has to has close geometry (so the example has three vertices.
    
    Double pharenteses allow for adding a "hole" in the geometry (as in the second example).
    

The examples are also pretty well-explained on Wikipedia:

    https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry
    
<h3> WKB - Well-known binary </h3>

Binary equivalent of WKT. Faster for reading and writing, but not readable.

<b> Excersise: </b> Try to create above geometry (of lineString) using WKT. WKT is written as a text " ". Use command .fromWKT()

<h2> Deleting objects </h2>

Only passing an index is necessary:


In [ ]:
layer.dataProvider().deleteFeatures([0])

<h2> DataProvider </h2>

A class that allows acessing geometry and attributes of objects of the layer. It has to be used to add, remove and edit features and their properties. Let's see how does it acts for objects editing:

<h2> Editing objects </h2>

Let's add two objects on a layer:

In [ ]:
layer = iface.activeLayer() #layer is currently active

feat1 = QgsFeature(layer.fields()) #new feature
feat1.setAttributes([0, 60]) #new attributes
line1_start = QgsPoint(15.74,50.89)
line1_end = QgsPoint(15.73,50.91)
feat1.setGeometry(QgsGeometry.fromPolyline([line1_start,line1_end])) #determining geometry of object 1

#object2
feat2 = QgsFeature(layer.fields())
feat2.setAttributes([0, 100])
line2_start = QgsPoint(11.74,53.89)
line2_end = QgsPoint(11.73,53.91)
feat2.setGeometry(QgsGeometry.fromPolyline([line2_start,line2_end]))


layer.dataProvider().addFeatures([feat1,feat2])  #adding objects to layer through dataProvider class

Now we will edit the object by changing its attributes:

In [ ]:
attrs = { 0 : 100, 1 : 90 } #flied 1 will be changed to 100, field 2 to 90
layer.dataProvider().changeAttributeValues({ 1 : attrs }) #object of index 1 will be changed to these values

We can also change geometry:

In [ ]:
geom = QgsGeometry.fromPolyline([QgsPoint(15.70,50.59),QgsPoint(15.70,50.52),QgsPoint(15.73,50.52)]) #we create new geometry of type QgsGeometry
layer.dataProvider().changeGeometryValues({1: geom}) #and substitute object of index 1 with new geometry

<h3> Better be cautious - editing mode </h3>

Changes made in editing mode can be reverted. Therefore it is better to made all the changes in editing mode:

    startEditing() - starts edit mode
    commitChanges() - confirm changes made
    rollBack() - revert to state before edit
    
    isEditable() - check if current layer is editable
    


<h3> New attrbiutes - how to add new attributes to the layer</h3>

dataProvider() has methods - addAttributes and deleteAttributes.

[New class] - QVariant, in QGIS is used to determine the data type of an attribute - integer, float etc.

In [ ]:
from qgis.PyQt.QtCore import QVariant #needs to be imported

layer.dataProvider().addAttributes\
([QgsField("name_string", QVariant.String), QgsField("name_int", QVariant.Int)]) #adding to attributes one of text type and second of integer type

In [ ]:
warstwa.dataProvider().deleteAttributes([0]) #deleting similarly to deleting features

In [ ]:
warstwa.updateFields() #Very important to ALWAYS udate fields after changes

<h3> Creating and writing layers </h3>

Layers writing is accomplished through $QgsVectorFileWriter$ class with its method named $writeAsVectorFormat()$, which supports all OGR data formats (GeoPackage, Shapefile, GeoJSON, KML and others)

In [ ]:
QgsVectorFileWriter.writeAsVectorFormat(layer, "nazwa", "UTF-8") #(layer, name, coding type (usally UTF-8)) 
#SHP by default

In [ ]:
QgsVectorFileWriter.writeAsVectorFormat(layer, "nazwa", "UTF-8", driverName="GeoJSON") #GeoJSON format

In [ ]:
crs=QgsCoordinateReferenceSystem("epsg:4326") #determine new coordinate system
QgsVecotFileWriter.writeAsVectorFormat(layer, "nazwa", "UTF-8", crs = crs, driverName="GeoJSON") #GeoJSON and in new coordinate system

<h3> Temporal layers </h3>

It is possible to create temporal layers which are not saved to hard drive and are deleted after project closing. They are useful to store some results.

Example:

In [ ]:
from qgis.PyQt.QtCore import QVariant

#empty layer in the memory
temp_vlayer = QgsVectorLayer("Point", "tymczasowe_punkty", "memory") #instead of "ogr" we pass an "memory" argument, instead of name we pass what data type will it store (for example "Point")
pr = temp_vlayer.dataProvider() #dataProvider of temporal layer

#Atributes - fields
pr.addAttributes([QgsField("name", QVariant.String), QgsField("age",QVariant.Int)])
temp_vlayer.updateFields() #ALWAYS do that

#Create object
feat = QgsFeature() #pusty obiekt
#Geometry
feat.setGometry(QgsGeometry.fromPointXY(QgsPointXY(10,10))) #a single point
#Atributes
feat.setAttributes(["Bobby",12])
#Add to layer
pr.addFeatures([feat])
temp_vlayer.updateExtents() #refresh layer

The layer is ready, has one object of determined attributes but it has to be added to the map to be displayed:


In [ ]:
QgsProject.instance().addMapLayer(temp_vlayer)

<h2> That's all for today! Now you are able to complete list 4! </h2>